# Lab 3 - Regression
- Structured API Approach

In [26]:
from pyspark.sql import SparkSession
import os

# Set JAVA_HOME explicitly
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-21-openjdk-amd64'

spark = SparkSession.builder \
    .appName("MySparkApp") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

In [27]:
spark   

In [28]:
test = spark.read.format('csv').load("/home/aaronpham/Coding/bigdata/spark/spark_mllib/data/nyc-taxi-trip-duration/test.csv", header=True, inferSchema=True)

train = spark.read.format('csv').load("/home/aaronpham/Coding/bigdata/spark/spark_mllib/data/nyc-taxi-trip-duration/train.csv", header=True, inferSchema=True)

In [29]:
sample = spark.read.format('csv').load("/home/aaronpham/Coding/bigdata/spark/spark_mllib/data/nyc-taxi-trip-duration/sample_submission.csv", header=True, inferSchema=True)

In [30]:
sample.show(2)

+---------+-------------+
|       id|trip_duration|
+---------+-------------+
|id3004672|          959|
|id3505355|          959|
+---------+-------------+
only showing top 2 rows


In [31]:
test.show(2)

+---------+---------+-------------------+---------------+------------------+-----------------+------------------+-----------------+------------------+
|       id|vendor_id|    pickup_datetime|passenger_count|  pickup_longitude|  pickup_latitude| dropoff_longitude| dropoff_latitude|store_and_fwd_flag|
+---------+---------+-------------------+---------------+------------------+-----------------+------------------+-----------------+------------------+
|id3004672|        1|2016-06-30 23:59:58|              1|-73.98812866210938|40.73202896118164|-73.99017333984375|40.75667953491211|                 N|
|id3505355|        1|2016-06-30 23:59:53|              1|-73.96420288085938|40.67999267578125|-73.95980834960938|40.65540313720703|                 N|
+---------+---------+-------------------+---------------+------------------+-----------------+------------------+-----------------+------------------+
only showing top 2 rows


In [32]:
test.printSchema()

root
 |-- id: string (nullable = true)
 |-- vendor_id: integer (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)



In [33]:
train.printSchema()

root
 |-- id: string (nullable = true)
 |-- vendor_id: integer (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- trip_duration: integer (nullable = true)



In [34]:
test.show(2)  

+---------+---------+-------------------+---------------+------------------+-----------------+------------------+-----------------+------------------+
|       id|vendor_id|    pickup_datetime|passenger_count|  pickup_longitude|  pickup_latitude| dropoff_longitude| dropoff_latitude|store_and_fwd_flag|
+---------+---------+-------------------+---------------+------------------+-----------------+------------------+-----------------+------------------+
|id3004672|        1|2016-06-30 23:59:58|              1|-73.98812866210938|40.73202896118164|-73.99017333984375|40.75667953491211|                 N|
|id3505355|        1|2016-06-30 23:59:53|              1|-73.96420288085938|40.67999267578125|-73.95980834960938|40.65540313720703|                 N|
+---------+---------+-------------------+---------------+------------------+-----------------+------------------+-----------------+------------------+
only showing top 2 rows


## Preprocess Data

In [35]:
# Replace the null value with average value
from pyspark.ml.feature import Imputer

feature_cols = ["pickup_longitude", "pickup_latitude", 
    "dropoff_longitude", "dropoff_latitude", 
    "passenger_count"]

imputer = Imputer(inputCols=feature_cols, outputCols=feature_cols)

imputer = imputer.fit(train)
train = imputer.transform(train)
test = imputer.transform(test)

train.show(3)

+---------+---------+-------------------+-------------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------+
|       id|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|  pickup_longitude|   pickup_latitude| dropoff_longitude|  dropoff_latitude|store_and_fwd_flag|trip_duration|
+---------+---------+-------------------+-------------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------+
|id2875421|        2|2016-03-14 17:24:55|2016-03-14 17:32:30|              1| -73.9821548461914| 40.76793670654297|-73.96463012695312|40.765602111816406|                 N|          455|
|id2377394|        1|2016-06-12 00:43:35|2016-06-12 00:54:38|              1|-73.98041534423828|40.738563537597656|-73.99948120117188| 40.73115158081055|                 N|          663|
|id3858529|        2|2016-01-19 11:35:24|2016-01-19 12:10:48|    

In [36]:
# Aggregate all the columns into one single feature column
from pyspark.ml.feature import VectorAssembler

features_assembler = VectorAssembler(inputCols=feature_cols, outputCol='features_cols')

train = features_assembler.transform(train)
test = features_assembler.transform(test)

train.show(2)

+---------+---------+-------------------+-------------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------+--------------------+
|       id|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|  pickup_longitude|   pickup_latitude| dropoff_longitude|  dropoff_latitude|store_and_fwd_flag|trip_duration|       features_cols|
+---------+---------+-------------------+-------------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------+--------------------+
|id2875421|        2|2016-03-14 17:24:55|2016-03-14 17:32:30|              1| -73.9821548461914| 40.76793670654297|-73.96463012695312|40.765602111816406|                 N|          455|[-73.982154846191...|
|id2377394|        1|2016-06-12 00:43:35|2016-06-12 00:54:38|              1|-73.98041534423828|40.738563537597656|-73.99948120117188| 40.73115158081055|               

## Train Decision Tree Regressor

In [37]:
from pyspark.ml.regression import DecisionTreeRegressor

dt = DecisionTreeRegressor(
    featuresCol="features_cols",
    labelCol="trip_duration",
    maxDepth=10   
)

dt_model = dt.fit(train)


In [38]:
predictions = dt_model.transform(test)
predictions.show(2)

+---------+---------+-------------------+---------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+------------------+
|       id|vendor_id|    pickup_datetime|passenger_count|  pickup_longitude|  pickup_latitude| dropoff_longitude| dropoff_latitude|store_and_fwd_flag|       features_cols|        prediction|
+---------+---------+-------------------+---------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+------------------+
|id3004672|        1|2016-06-30 23:59:58|              1|-73.98812866210938|40.73202896118164|-73.99017333984375|40.75667953491211|                 N|[-73.988128662109...| 749.5248047836944|
|id3505355|        1|2016-06-30 23:59:53|              1|-73.96420288085938|40.67999267578125|-73.95980834960938|40.65540313720703|                 N|[-73.964202880859...|1040.6451612903227|
+---------+---------+-------------------+----

## Evaluation

In [39]:
# # Join predictions with sample to get actual trip_duration values
# predictions_with_actual = predictions.join(sample, on="id", how="left")
# predictions_with_actual.show(5)